* **Introduction :**

This notebook meant for giving a look a bit closer inside the contents of this kind of malls as a high view. I tried to figure out the different type of datasets exist in the competetion, by expaling some relations between different datasets, and attempting to understand the distribution of data and the importance of each branch...etc.

The explanation will come out in different ways, interpretaion, tables, histograms, images and curves. 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os  # manipulating with directory for uploading data 
import matplotlib.pyplot as plt  # ploting curves and figures 
from matplotlib import style
import seaborn as sns # ploting curves and figures 
import glob
import matplotlib.image as mpimg  # images visulizations 
from collections import Counter  # counts operations
import operator
from PIL import Image

style.use('fivethirtyeight')

***# Importing data from articles.csv to handle and deal with the contents***

In [ ]:
article_id = pd.read_csv ('../input/h-and-m-personalized-fashion-recommendations/articles.csv')
real_data= article_id.drop('detail_desc', axis=1).copy()

**# Getting closer look at the contents**

In [ ]:
article_id.info() 
# this info declare zeros null and no missing data

In [ ]:
# real_data.description
real_data.describe()

In [ ]:
# checking out the rows duplications
dupl = real_data.duplicated()
if dupl.any():
    print(dupl)
else: print('Data not duplicated')

In [ ]:
# we count the amount of each content separatly 
def data_count():
    data_counts = {}
    for col in real_data.columns:
        difrnt_data = real_data[col].unique()
        data_counts[col] = difrnt_data, len(difrnt_data) # counting the unique element in each columns
 
    df = pd.DataFrame([[i, data_counts[i][-1]] for i in data_counts], columns=['items_name','contents_nbrs'])
    df.set_index('items_name')
    return df, data_counts
    # Deppartment_handling
df, data_counts= data_count()
df

*  *in the next sections we will go ahead and check the distribution of objects within section and departments in general, and make some comparison among departments, section and indexes*

In [ ]:
def dep_sec_Gr(reference_col, class_1, class_2, items_1=False, items_2=False, department_content=False, products_types=False):
    
    rows_list = []
    for row in real_data[reference_col].unique():
        # importing data for each class depensding on the referenced column
        frst_content = real_data[class_1][(real_data[reference_col] == row)] 
        scnd_content = real_data[class_2][real_data[reference_col] == row]

        if items_1:
            rows_list.append([row,len(frst_content.unique()), len(scnd_content.unique()), frst_content.unique(),
                                         scnd_content.unique()])
            department_content = pd.DataFrame(rows_list, columns=[item for item in items_1]) # dataframe for departments data

        if items_2:
            rows_list.append([row, len(frst_content.unique()), len(scnd_content.unique()), scnd_content.unique()])
            products_types = pd.DataFrame(rows_list, columns=[item for item in items_2]) # dataframe for sections data

    return department_content, products_types

dep, _ = dep_sec_Gr(reference_col='department_name', class_1='section_name', class_2='index_name',
           items_1=['departments_name', 'sections_amount', 'index_amount','sections_name', 'index_names', ])

_, sec = dep_sec_Gr(reference_col='section_name', class_1='index_name', class_2='product_group_name',
               items_2=['section_name', 'index_amount', 'groups_amount', 'groups_name'])

In [ ]:
# let's find out departs with sects and indexs
dep

In [ ]:
# let's find out sections with indexs and groups
sec

In [ ]:
# visulization 10 biggest dapartments by the amount of contents 

bgst_depa = Counter(real_data['department_name']) # counting the amount of each element in each dpart
bgst_depa = Counter(bgst_depa).most_common(10) # choosing the biggest 10 departs

# creating plot
x = list(map(lambda x: x[0] , (bgst_depa)))
y = list(map(lambda x: x[1] , (bgst_depa)))

fig = plt.figure(num=None, figsize=(15, 5), dpi=80)

ax = sns.barplot(x=x, y=y) 

for cont in ax.containers:
    ax.bar_label(cont)
    
ax = ax.set_xticklabels(ax.get_xmajorticklabels(), rotation=70)
plt.locator_params ('y', nbins = 10)

plt.title('biggest_department_per_content')
plt.show()

In [ ]:
# let's see the most 5 product types in each product group
product_group = real_data['product_group_name'].unique()
mst5_products= list()
for i in product_group:
    product_type = Counter(real_data['product_type_name'][(real_data['product_group_name'] == i)])

    mst5_products.append(product_type.most_common(5)) # choosing the most 5 products types

df = pd.DataFrame([mst5_products[i] for i in range(len(mst5_products))], index=product_group[:], columns=[1, 2, 3, 4, 5])
mst5_products_types = df.T
mst5_products_types

In [ ]:
# preaparing for importing and printing out the related images to 
# one sample of the previous mst5 list 

article_id=[]
for item in df[1]:
    article_id.append(real_data['article_id'][(real_data['product_type_name'] == item[0])]) # importing the ids of each type to prepare it for visualiztion
    

# preparing the paths of each image from images file
imgs=[]  
exist = False
for idx in range(len(article_id)):
    for value in article_id[idx].values:
        for items in glob.glob('/kaggle/input/h-and-m-personalized-fashion-recommendations/images/*'):
            for img in os.listdir(items):
                item = int(img.split('.')[0])
                if item == value:
                    imgs.append("{}/{}".format(items, img))
                    exist= True
                    break
        if exist:
            exist= False 
            break

# listing the products images names            
types = list(map(lambda x: x[0], df[1]))

# listing the existed iamges
y = [img for img in imgs]

# starting the plotting
plt.figure(figsize=(15, 10))

for i in range(len(imgs)):
    plt.subplot(5, 5, i+1)
    image = Image.open(y[i])
    plt.imshow(image) 
    plt.axis('off')
    plt.gca().set_title(types[i]) 
    
plt.show()

* We will handle the objects this time according to the transactions that we get
* The method depends on reading, distributing and visulizing the datasets per **years** *

In [ ]:
# importing the transactions file

trans = pd.read_csv('/kaggle/input/h-and-m-personalized-fashion-recommendations/transactions_train.csv')

In [ ]:
trans.head(5)

In [ ]:
# cheking out the null and duplication in transactions file
null = trans.isnull()
dup = trans.duplicated()

In [ ]:
for i, nl in enumerate(null.any()):
    if nl:
        print(nl)
    else:print('trans row {} witout null'.format(i+1))
if not dup.any():
    print(dup)
else:print('trans witout duplication')

In [ ]:
data = pd.DatetimeIndex(trans['t_dat']).year
trans['t_dat'] = pd.to_datetime(trans.t_dat, format='%Y-%m-%d')

In [ ]:
# extracting the studied years and their months 
years = trans['t_dat'].dt.year.unique()
print(years)
for year in years:
    month = trans['t_dat'][(trans['t_dat'].dt.year==year)].dt.month.unique()
    print(f'{year}:',month)


In [ ]:
# the amount and prices of sales in each year
 
def determine_year(years): # amin function for importing data based on wanted year
    trans_year = trans[(trans['t_dat'].dt.year==years)].copy()
    return trans_year

def price_nbr_count(years): # counting the price and nbr of sales
    trans_year = determine_year(years)
    nbr_sales = len(trans_year.article_id)
    sales_price = sum(trans_year.price)
    
    return nbr_sales, np.round(sales_price, 2)

nbr_sales_18, sales_price_18 = price_nbr_count(2018)
nbr_sales_19, sales_price_19 = price_nbr_count(2019)
nbr_sales_20, sales_price_20 = price_nbr_count(2020)

In [ ]:
price_nbr_counts = pd.DataFrame([['2018',nbr_sales_18, sales_price_18],['2019',nbr_sales_19,sales_price_19],
                                 ['2020',nbr_sales_20,sales_price_20]], columns=['years','sales_amount','sales_toatal_price'])
price_nbr_counts

In [ ]:
# function for counting the best sales per how much and how cost 

def most_sales_price(year):
    trans_year = determine_year(year)
    
    # mst_items
    count_articles = Counter(trans_year.article_id)
    articles_id_nbr = count_articles.most_common(20)
    
    # sum of price
    articles_price = {}
    for item in count_articles: 
        article_pr = trans_year['price'][trans_year['article_id']==item]
        articles_price[item] = sum(article_pr)
    
    articles_id_nbr = count_articles.most_common(20)
    ids_nbr = [real_data.prod_name.values[real_data.article_id==item[0]] for item in articles_id_nbr] # most 20 prod names
    
    sorted_bst = sorted(articles_price.items(), key=operator.itemgetter(1))[-20:]
    ids_bst = [real_data.prod_name.values[real_data.article_id==item[0]] for item in sorted_bst] # sorting the bst prod names by price 

    # dataframe for most and best sales 
    articles_bst = pd.DataFrame([[x[0], ids_bst[idx][0], x[1], articles_id_nbr[idx][0], ids_nbr[idx][0], articles_id_nbr[idx][1]]for idx, x in enumerate(sorted_bst)], index=[np.arange(1, len(sorted_bst)+1)], 
                            columns=('bst_artic', 'bst_prod_name','bst_prices','mst_artc', 'mst_prod_name','mst_artc_amount'))
    articles_bst.sort_index(inplace=True)

    return count_articles, articles_bst


In [ ]:
# this function meant for showing out the iamges of the sales per quantity and prices 
def img_visu(bst_art, prod_name):
    imgs=[]  
    titles = []
    exist = False
    for i, value in enumerate(bst_art):
        for items in glob.glob('/kaggle/input/h-and-m-personalized-fashion-recommendations/images/*'):
            for img in os.listdir(items):
                item = int(img.split('.')[0])
                if item == value:
                    imgs.append("{}/{}".format(items, img))
                    titles.append(prod_name[i+1])
                    exist= True
                    break
            if exist:
                exist = False
                break
        
    # creating plot
    y = [img for img in imgs]

    plt.figure(figsize=(15, 15))
    for i in range(len(imgs)):
        plt.subplot(5, 5, i+1)
        image = Image.open(y[i])
        plt.imshow(image) 
        plt.axis('off')
        plt.gca().set_title(titles[i], size=15) 

    plt.show()


In [ ]:
# this function visualizes the the departments with most number of sales and their percentage according to the whole amount of sales 
# we will just show the deprtments as it's the highest levle in malls  
def bst_depart_sales(count_articles, year):
    # preparing best departments per their sales amount
    dprtmnt = {}
    for i in count_articles.items():  
        dep = np.array(real_data.department_name[real_data.article_id==i[0]].values)
        dep = dep.all()
        if dep in dprtmnt:
            dprtmnt[dep] = dprtmnt[dep]+i[1]
        else:
            dprtmnt[dep] = i[1]
    dep_bst = sorted(dprtmnt.items(), key=operator.itemgetter(1))[-20:]

    # Creating plot
    total = []
    for item in dprtmnt:
        total.append(dprtmnt[item])

    x=[item[0] for item in dep_bst]
    percentage = [np.round(item[1] / sum(total)*100, 2) for item in dep_bst]

    fig = plt.figure(num=None, figsize=(15, 4), dpi=80)

    ax = sns.barplot(x=x, y=percentage) 
    for cont in ax.containers:
        ax.bar_label(cont, fmt='%.1f%%')
    ax = ax.set_xticklabels(ax.get_xmajorticklabels(), rotation=70)

    plt.title('best 20_departements per sales amount for {}'.format(year))
    plt.show()

In [ ]:
# Finally we will see the progress of sales during the years per the sum of prices each week 

def sales_price_progression(year):
    # distributing sales pries per week 
    weeks = [w for n, w in trans[trans['t_dat'].dt.year==year].groupby(pd.Grouper(key='t_dat',freq='W'))]
    weekly_prices = [np.sum(weeks[idx]['price']) for idx in range(len(weeks))]
    
    # creating plot
    x = range(len(weeks))
    y = weekly_prices

    fig = plt.figure(figsize=(15,7))

    plt.plot(x, y, color='r', scalex=y, linewidth=2, marker='*', linestyle=(0, (5, 2, 1, 1)), label='progression line')
    plt.xlabel('weeks')
    plt.ylabel('weekly_prices')
    
    plt.locator_params ('x', nbins = 30)
    plt.locator_params ('y', nbins = 20)
    
    plt.title('sales movement progression for {}'.format(year))
    plt.legend()

    plt.show()


* ***2018 year anlysis***

In [ ]:
# importing the amount of sales and the prices
count_articles, articles_price = most_sales_price(2018)

In [ ]:
# showing images of articles_id with most sales
img_visu(articles_price.mst_artc, articles_price.mst_prod_name)

In [ ]:
# showing images of articles_id with most prices
img_visu(articles_price.bst_artic, articles_price.bst_prod_name)

In [ ]:
# visualizing biggest departs per amount of sales
bst_depart_sales(count_articles, 2018)

In [ ]:
# progression of sales movement during the year
sales_price_progression(2018)

* ***2019 year anlysis***

In [ ]:
# importing the amount of sales and the prices
count_articles, articles_price = most_sales_price(2019)

In [ ]:
# showing images of articles_id with most sales
img_visu(articles_price.mst_artc, articles_price.mst_prod_name)

In [ ]:
# showing images of articles_id with most prices
img_visu(articles_price.bst_artic, articles_price.bst_prod_name)

In [ ]:
# visualizing biggest departs per amount of sales
bst_depart_sales(count_articles, 2019)

In [ ]:
# progression of sales movement during the year
sales_price_progression(2019)

* ***2020 year anlysis***

In [ ]:
# importing the amount of sales and the prices
count_articles, articles_price = most_sales_price(2020)

In [ ]:
# showing images of articles_id with most sales
img_visu(articles_price.mst_artc, articles_price.mst_prod_name)

In [ ]:
# showing images of articles_id with most prices
img_visu(articles_price.bst_artic, articles_price.bst_prod_name)

In [ ]:
# visualizing biggest departs per amount of sales
bst_depart_sales(count_articles, 2020)

In [ ]:
# progression of sales movement during the year
sales_price_progression(2020)

This is the ending of analyzing the objects and sales according to the amount of objects and their distributions among the departments and sections, and the movement of sales during the years of transactions.